In [1]:
##LIBRARIES
import pandas as pd
import numpy as np
import seaborn as sns
import os
import sys
import glob
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix 
from psynlig import plot_correlation_heatmap
import pingouin as pg
from scipy.stats import pearsonr
import plotly.express as px
import kaleido

In [2]:
path = ('Results_cor_GSD1A_2023MS') 
if not os.path.exists(path):
    os.mkdir(path)
    print('Output folder created')

Output folder created


In [ ]:
all_files = glob.glob('Uri-integration/*.csv')

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
df["Sample"] = df["Cell Type"].astype(str) + df["CELL ID "].astype(str) + df["COMPOUND"].astype(str)
df.set_index(["Sample"], inplace = True,
                             append = True, drop = False)
df.fillna(0, inplace=True)\
df2 = df.groupby(level=1).mean()

In [6]:
df_clean = df2.drop(columns=['CELL ID '])
df_normalbymean=(df_clean-df_clean.mean())/df_clean.std()

In [9]:
## export for R vis 
df_normalbymean.to_excel(path + '//' + 'networkanalysis.xlsx')
print("done")

done


In [ ]:
## corelation 

In [ ]:
df = pd.read_csv(r'GSD1A_corelation_data/COR_DATA.csv')

In [ ]:
df["Sample"] = df["Cell Type"].astype(str) + df["CELL ID "].astype(str) + df["COMPOUND"].astype(str)
df.set_index(["Sample"], inplace = True,
                             append = True, drop = False)
df.fillna(0, inplace=True)
df.set_index(["Sample"], inplace = True,
                             append = True, drop = False)

In [ ]:
df2 = df.groupby(level=1).mean()
df_clean = df2.drop(columns=['CELL ID '])



In [ ]:
##norm1
df_normalbymean=(df_clean-df_clean.mean())/df_clean.std()

In [ ]:
##norm2
df_max_scaled = df_clean.copy()
for column in df_max_scaled.columns:
    df_max_scaled[column] = df_max_scaled[column]  / df_max_scaled[column].abs().max()

In [ ]:
##norm3
df_min_max_scaled = df_clean.copy()
for column in df_min_max_scaled.columns:
    df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())   

In [ ]:
##list frames
corleation_list_all = [(df_normalbymean), (df_max_scaled), (df_min_max_scaled)]

In [ ]:
# define cor function
def cor_function(list):
      for  x in list:
        cor = x.corr(method = 'pearson').round(2)
        sns.heatmap(cor, annot=True, vmax=1, vmin=-1, center=0, cmap='vlag')
        sns.set_style("dark")
        cax = plt.gcf().axes[0]
        cax.tick_params(labelsize=10)
        plt.yticks(rotation="horizontal", fontsize=6)
        plt.xticks(rotation="vertical", fontsize=6)
        plt.margins(0.2)
        plt.subplots_adjust(bottom=0.4)
        name = [i for i in globals() if globals()[i] is x][0]
        plt.savefig(path + '\\' + '{}pearsonheatmap2_full-tryP.pdf'.format(name))
        plt.close()
        cor = cor.unstack()
        cor = abs(cor)
        writer = pd.ExcelWriter(path + '//' + '{}PCORDATA-excel.xlsx'.format(name), engine='xlsxwriter')
        cor.to_excel(writer)
        writer.save()
        print('done:)')
        rho = x.corr()
        pval = x.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
        p = pval.applymap(lambda x: ''.join(['*' for t in [0.0001,0.001,0.01,0.1,] if x<=t]))
        try_1 = rho.round(2).astype(str) + p
        try_1.to_html(path + '\\' + '{}try1.html'.format(name))
        print('done:)2')

In [ ]:
def bubble_plot(list):
    for x in list:
        kwargs = {
    'heatmap': {
        'vmin': -1,
        'vmax': 1,
        'cmap': 'viridis',
    },
    'figure': {
        'figsize': (12,8),
    },
    }
        name = [i for i in globals() if globals()[i] is x][0]
        plot_correlation_heatmap(x, bubble=True, annotate=True,textcolors=['white', 'black'],
                                 val_fmt='{x:.2f}',  **kwargs)
        plt.style.use("classic")
        cax = plt.gcf().axes[0]
#         cax.tick_params(labelsize=3)
#         plt.yticks(rotation="horizontal", fontsize=6)
#         plt.xticks(rotation="horizontal", fontsize=6)
        font = {'family' : 'Arial',
        'size'   : 10}

        plt.rc('font', **font)
# plt.margins(0.1)
# plt.subplots_adjust(bottom=0.3)
        plt.savefig(path + '\\' + '{}bubblescatmatrixv2.pdf'.format(name), dpi=600)
        plt.show()
        print('done:)')


In [ ]:
cor_function(corleation_list_all)
bubble_plot(corleation_list_all)

In [ ]:
####OMICS INTEGRATION VIS 

In [ ]:
path = pd.read_csv(r'enrichr_pathways.csv')

In [ ]:
fig = px.bar(path, x="log10(adj.p_val)", y="Term", orientation='h',
            color='Combined Score', height=400,
            labels={"Term": "pathway"}
             # add day column to hover data
           )
fig.update_layout(
            font_family="Arial",
            font_color="Black",
            font_size=8,
            font=dict(
                family="Arial",
                size=10,
                color="Black"))
fig.show()
fig.write_image("enrichr.pdf", engine="kaleido")

In [ ]:
##################################################